# The basics
This notebook explains 2 basic principles for writing effective prompts along with a selection of tactics that can be used to accomplish these principles

## Setup

Import necessary libraries and load openAI API key

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

openai.api_key = os.getenv('OPENAI_API_KEY')

#### Helper function to get GPT response

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
  messages = [{"role": "user", "content": prompt}]
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    temperature=0 # degree of randomness of the model's output
  )
  return response.choices[0].message["content"]

## Prompting Principles

### Principle 1: Write clear and specific instructions

#### Tactics

##### Tactic 1: Use delimiters to clearly indicate distinct parts of the input
For e.g. if you are asking the model to summarize a text, may be you copied and pasted the content from some external source without reading it. And the text you are providing also sounds like a task. The model might get confused, which one is the actual task? the one you wrote or the one you copy pasted. Because both the text sound like a task to be done! So it is always a good idea to separate the input content from the task being given using delimiters for clarity.
- Delimiters can be anything like: ```, """, ''', <>, <tag></tag>, : etc.

In [ ]:
text = f"""
You should express what you want a model to do by providing instructions that are clear and specific as you can possibly make them.
This will guide the model towards the desired output, and reduce chances of receiving irrelevant or incorrect responses.
Don't confuse writing a clear prompt with writing a short prompt. In many cases, longer prompts provide more clarity and context
for the model, which can lead to more detailed and relevant outputs.
"""

prompt = f"""
Summarize the text delimited by triple backticks into a single sentence.
```{text}```
"""

response = get_completion(prompt)
print(response)

In [ ]:
prompt = f"""
Summarize the text delimited by triple backticks into a single sentence in less than 20 words.
```{text}```
"""

response = get_completion(prompt)
print(response)

##### Tactic 2: Ask for a structured output
- JSON, html, xml

In [ ]:
prompt = f"""
Generate a list of three made-up movie titles from the star wars franchise along
with their star cast, director and genre. Provide them in JSON format with the following keys:
movie_id, title, star_cast, director, genre.
"""
response = get_completion(prompt)
print(response)

##### Tactic 3: Ask the model to check whether conditions are satisfied

In [ ]:
text = f"""
Making a cup of tea is easy! First, you need to get some water boiling. While that's happening,
grab a cup and put a tea bag in it. Once the water is hot enough, just pour it over the tea bag.
Let it sit for a bit so the tea can steep. After a few minutes, take out the tea bag. If you like,
you can add some sugar or milk to taste. And that's it! You've got yourself a delicious cup of tea to enjoy.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. If it contains a sequence of instructions,
re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"

\"\"\"{text}\"\"\"
"""

response = get_completion(prompt)
print(response)

In [ ]:
text = f"""
The sun is shining brightly today, and the birds are singing. It's a beautiful day to go for a
walk in the park. The flowers are blooming, and the trees are swaying gently in the breeze. People
are out and about, enjoying the lovely weather. Some are having picnics, while others are playing
games or simply relaxing on the grass. It's a perfect day to spend time outdoors and appreciate the
beauty of nature.
"""

prompt = f"""
You will be provided with text delimited by triple quotes. If it contains a sequence of instructions,
re-write those instructions in the following format:

Step 1 - ...
Step 2 - ...
...
Step N - ...

If the text does not contain a sequence of instructions, then simply write \"No steps provided.\"

\"\"\"{text}\"\"\"
"""

response = get_completion(prompt)
print(response)

##### Tactic 4: Ask to respond in give style/tone

In [ ]:
prompt = f"""
Your task is to answer in a consistent style.

<child>: Teach me about patience.

<grandparent>: The river that carves the deepest valley flows from a modest spring; the 
grandest symphony originates from a single note; the most intricate tapestry begins with a solitary thread.

<child>: Teach me about resilience.
"""
response = get_completion(prompt)
print(response)

### Principle 2: Give the model time to think

If you don't give the model time to think, it might give wrong answers just like at an interview if you ask the candidate to solve a hard problem and wont give him time, or give him less time, he'll mess it up in hurry, but he might be able to solve it if given time to think.

##### Tactic 1: Split the task into multiple steps and specify the steps required to complete the task
This way the model will split the task into multiple steps and take longer to respond, but it might respond more accurately

In [ ]:
text = f"""
In a charming village, siblings Jack and Jill set out on a quest to fetch water from a hilltop
well. As they climbed, singing joyfully, misfortune struck—Jack tripped on a stone and tumbled
down the hill, with Jill following suit. Though slightly battered, the pair returned home to
comforting embraces. Despite the mishap, their adventurous spirits remained undimmed, and they
continued exploring with delight.
"""
# example 1
prompt_1 = f"""
Perform the following actions: 
1 - Summarize the following text delimited by triple backticks in 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following keys: french_summary, num_names.

Separate your answers with line breaks.

Text:
```{text}```
"""

response = get_completion(prompt_1)
print(response)

**Ask for the output in a specific format.**  
Here we'll provide the same above text as input, similar steps, but a different output format

In [ ]:
prompt_2 = f"""
Your task is to perform the following actions: 
1 - Summarize the following text delimited by <> with 1 sentence.
2 - Translate the summary into French.
3 - List each name in the French summary.
4 - Output a json object that contains the following keys: french_summary, num_names.

Use the following format:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in French summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print(response)

##### Tactic 2: Specifically instruct the model to work out it's own solution before rushing to a conclusion

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

The model responds, *student's solution is correct*
> Notice, the student's solution is not actually correct 💩

##### We can fix this by instructing the model to work out it's own solution first, before checking the student's solution

In [ ]:
prompt = f"""
Your task is to determine if the student's solution is correct or not.
To solve the problem do the following:
- First, work out your own solution to the problem. 
- Then compare your solution to the student's solution and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
I'm building a solar power installation and I need help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
``` 
Student's solution:
```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
Actual solution:
"""
response = get_completion(prompt)
print(response)

## Model Hallucinations 😵‍💫
These models are sometimes eating mushrooms 🍄, they hallucinate things, they'll tell you things which are not real

Look at the following example, Tesla is a real company, but the product is not real

In [ ]:
prompt = f"""
Tell me about AeroGlide UltraSlim Smart Toothbrush by Boie
"""
response = get_completion(prompt)
print(response)

This prompt, gives information about the toothbrush as if it really exists.
> Remember, the models have not memorized the entire internet.
We can work around this problem, by asking the model to first find the relevant information, and then answer the question based on the relevant information it found. And do not answer question until it finds the required relevant information.